# Spatial Mapping

This notebook contains the code for and the final product of the active new building construction site scoping project&mdash;a geo-map of building lots still under construction in New York City.

## Introduction

### Classifying active construction lots

A lot is considered to be a building under construction, for the purposes of this project, if it has been issued a DOB construction permit which has not yet expired, but no certificate of occupancy is on record with the DOB with a date of granting that is more recent than the construction permit approval date.

Work can begin as soon as a construction permit is issued; permits usually last one year before they expire and require renewal. A building cannot actually be occupied until all inspections have taken place and a certificate of occupancy has been issued&mdash;thus the "C of O" approval is traditionally seen as the end of a construction project.

The single caveat is when a lot has recieved a construction permit but due to issues on the side of the permitee that lot never entered construction. Since permits expire after a year, unless a permitee is constantly renewing permits and not doing work I expect that this error is small.

It's also a bit of a philosophical question as to whether or not a lot which is effectively just an unused hole-in-the-ground is a construction site, an active construction site, or just a hole-in-the-ground.

Our classification is as tight as it can be, with respect to what the city regulates.

### Data processing

I use two data sources for this project. The first data source is the [DOB permit issuance dataset](https://data.cityofnewyork.us/Housing-Development/DOB-Permit-Issuance/ipu4-2q9a) on NYC Open Data, from which I retrieve a list of lots with non-expired construction permits (as of writing). The second data source is the DOB [BISweb interface](http://a810-bisweb.nyc.gov/bisweb/bsqpm01.jsp), which provides, in part, PDF copies of the certificates of occupancy that DOB has on its digital record (all recent ones; certificates going further back are more tenuous, with some document scans reaching back as far as ~1900). These are scraped using the `co_reader` module, a Python module that was written for the topically similar [NYC Construction Timeline project](https://github.com/ResidentMario/nyc-construction-timeline) which uses a text scanner to parse out dates from issued certificates.

This data processing is handled in the preceding `Active New Building Construction Site Data Join.ipynb` notebook.

### Dataset

At this point we have a file, `Active New Building Construction Sites.csv`, which contains a unified recordset of all of the active under-construction BINs in New York City (as well as some data about them, taken from the original permit dataset). This file is the source of the map generated by the notebook code in this file.

### Further work

The end product of this notebook is a mapping of active construction sites in New York City, as of the date of processing in mid-July 2016. However, this is functionally no more than a proof of concept. Constructing the dataset required reading in 4000 BISweb certificates, a herculean process that does not scale to operationalization as it requires probably over 24 hours overall to run. A finer-resolution active construction sites chart generated on a daily, monthly, or weekly basis would require an easier way to access this data (and one that wouldn't load down the DOB servers every time it is requested).

I suggest an Open Data dataset of certificate of occupancy releases.

### Small note

BIN `1012420`, address `507 WEST 28 STREET MANHATTAN`, has [65 replicitious certificates of occupancy](http://a810-bisweb.nyc.gov/bisweb/COsByLocationServlet?requestid=4&allbin=1012420), somehow. It correspond with a massive new apartment tower on the High Line ([source](https://newyorkyimby.com/category/507-west-28th-street), [source](http://www.6sqft.com/west-chelseas-tallest-tower-rises-and-finally-reveals-itself/), et. al.). Interesting shape issue.

## Test map construction

The data processing necessary to get this dataset takes a long time. While I waited I used the sample from the first of the 40 runs of the script to test-drive the map.

In [1]:
import geopandas as gpd
import pandas as pd
import requests
import zipfile
import io
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option("max_columns", 500)

### Reading geospatial lot data

The MapPLUTO datasets are beefy ones, provided as seperate zipfiles for each of the boroughs. The following lines of code localize that data.

Note that these files have a high memory requirement.

In [2]:
r = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/bk_mappluto_16v1.zip')
with zipfile.ZipFile(io.BytesIO(r.content)) as ar:
#     ar.extract('BKMapPLUTO.shp')
#     ar.extract('BKMapPLUTO.shx')
    ar.extractall("data/brooklyn/")
del r

In [ ]:
r = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/bx_mappluto_16v1.zip')
with zipfile.ZipFile(io.BytesIO(r.content)) as ar:
#     ar.extract('BXMapPLUTO.shp')
#     ar.extract('BXMapPLUTO.shx')
    ar.extractall("data/bronx/")
del r

In [ ]:
r = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/mn_mappluto_16v1.zip')
with zipfile.ZipFile(io.BytesIO(r.content)) as ar:
#     ar.extract('MNMapPLUTO.shp')
#     ar.extract('MNMapPLUTO.shx')
    ar.extractall("data/manhattan/")
del r

In [ ]:
r = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/qn_mappluto_16v1.zip')
with zipfile.ZipFile(io.BytesIO(r.content)) as ar:
#     ar.extract('QNMapPLUTO.shp')
#     ar.extract('QNMapPLUTO.shx')
    ar.extractall("data/queens/")
del r

In [ ]:
r = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/si_mappluto_16v1.zip')
with zipfile.ZipFile(io.BytesIO(r.content)) as ar:
#     ar.extract('SIMapPLUTO.shp')
#     ar.extract('SIMapPLUTO.shx')
    ar.extractall("data/staten_island/")
del r

In [3]:
brooklyn = gpd.read_file("data/brooklyn/", layer='BKMapPLUTO')

In [4]:
brooklyn.head(5)

,APPBBL,APPDate,Address,AllZoning1,AllZoning2,AreaSource,AssessLand,AssessTot,BBL,BldgArea,BldgClass,BldgDepth,BldgFront,Block,BoroCode,Borough,BsmtCode,BuiltCode,BuiltFAR,CB2010,CD,CT2010,ComArea,CommFAR,CondoNo,Council,EDesigNum,Easements,ExemptLand,ExemptTot,Ext,FacilFAR,FactryArea,FireComp,GarageArea,HealthArea,HistDist,IrrLotCode,LandUse,Landmark,Lot,LotArea,LotDepth,LotFront,LotType,LtdHeight,MAPPLUTO_F,NumBldgs,NumFloors,OfficeArea,OtherArea,Overlay1,Overlay2,OwnerName,OwnerType,PLUTOMapID,PolicePrct,ProxCode,ResArea,ResidFAR,RetailArea,SHAPE_Area,SHAPE_Leng,SPDist1,SPDist2,Sanborn,SanitBoro,SanitDist,SanitSub,SchoolDist,SplitZone,StrgeArea,TaxMap,Tract2010,UnitsRes,UnitsTotal,Version,XCoord,YCoord,YearAlter1,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0.000000e+00,None,1081 EAST 12 STREET,R5,None,2,35107.0,91336.0,3.067140e+09,2112,C3,48.0,22.00,6714,3,BK,5,None,0.60,3000,314,534,0,0.0,0,44,None,0,2360.0,2360.0,None,2.0,0,L156,0,7310,None,N,02,None,55,3500,100.0,35.00,5,None,0,1,2.0,0,0,None,None,"RAMBOD, SHAHROKH",None,1,70,0,2112,1.25,0,3421.921447,270.323044,None,None,313 040,3,14,4D,21,N,0,32006,0534,4,4,16v1,994376,166232,0,0,1931,None,11230,R5,None,None,None,22d,"POLYGON ((994429.5163999945 166222.5835999995,..."
1,3.035210e+09,10/23/1996,157 CHESTER STREET,C4-3,None,2,16827.0,24718.0,3.035210e+09,2955,B2,50.0,19.58,3521,3,BK,1,None,1.51,1005,316,924,0,3.4,0,41,None,0,1550.0,1550.0,None,4.8,0,L120,0,5900,None,N,01,None,112,1958,100.0,19.58,5,None,0,1,2.0,0,0,None,None,JOYCE MILLER,None,1,73,3,2955,2.43,0,2017.827636,240.532209,None,None,316 053,3,16,2A,23,N,0,31201,0924,2,2,16v1,1008947,182627,0,0,1995,None,11212,C4-3,None,None,None,17d,"POLYGON ((1008998.082800001 182627.6992000043,..."
2,0.000000e+00,None,65 EAST 95 STREET,R6,None,2,12291.0,64350.0,3.045980e+09,3502,C3,85.0,20.50,4598,3,BK,5,E,1.08,1000,317,884,0,0.0,0,41,None,0,0.0,0.0,G,4.8,0,E283,0,5810,None,N,02,None,64,3250,130.0,25.00,5,None,0,2,2.0,0,0,None,None,SYLVIA GWENDOLYN ASH,None,1,67,0,3502,2.43,0,3117.126286,307.209614,None,None,316 041,3,17,1A,17,N,0,31501,0884,4,4,16v1,1004844,181200,0,0,1930,None,11212,R6,None,None,None,17b,"POLYGON ((1004889.778200001 181247.4655999988,..."
3,0.000000e+00,None,156 MOFFAT STREET,R6,None,2,4716.0,17464.0,3.034460e+09,2160,B1,54.0,20.00,3446,3,BK,2,E,1.08,2002,304,411,0,0.0,0,37,None,0,1550.0,1550.0,None,4.8,0,Q252,0,3500,None,N,01,None,27,2000,100.0,20.00,5,None,0,1,2.0,0,0,None,None,ANTHONY ROBERTS,None,1,83,3,2160,2.43,0,2007.615215,241.745463,None,None,309 021,3,04,3A,32,N,0,31109,0411,2,2,16v1,1010124,189540,0,0,1910,None,11207,R6,None,None,None,17c,"POLYGON ((1010170.660400003 189517.8780000061,..."
4,0.000000e+00,None,2228 79 STREET,R5,None,2,7070.0,29859.0,3.062770e+09,1442,B9,39.5,19.50,6277,3,BK,1,E,0.72,1001,311,270,0,0.0,0,44,None,0,1550.0,1550.0,None,2.0,0,E253,0,8300,None,N,01,None,18,2000,100.0,20.00,5,None,0,1,1.0,0,0,None,None,"GUAN, HUI TING",None,1,62,3,1442,1.25,0,1944.642241,237.284617,None,None,312 075,3,11,5C,21,N,0,31904,0270,2,2,16v1,987277,159781,0,0,1940,None,11214,R5,None,None,None,22d,"POLYGON ((987316.2714000046 159814.2085999995,..."


### Reading city outline shapefile

Following Deena's lead on this, I am using the [NYC City Council GeoJson](https://github.com/dwillis/nyc-maps) from the public `nyc-maps` repository.

In [47]:
r = requests.get('https://raw.githubusercontent.com/dwillis/nyc-maps/master/city_council.geojson')
with open("NYC Community Districts.geojson", "w") as f:
    f.write(r.text)
del r

<!-- 

### Geocoding BINs into BBLs (test)

All of the data provided by the DOB is in resolution of BINs, while the data provided by MapPLUTO is in terms of BBLs.

There are a couple of different ways around this. The first option is to use instead the [buildings footprint dataset](https://data.cityofnewyork.us/Housing-Development/Building-Footprints/nqwf-w8eh), a recently-released, BIN-classified dataset of building footprints that was composed by drone overflight (let that sink for a moment). However, this was done in 2014, so the data is probably out of date. MapPLUTO, by contrast, is released yearly, and in fact was most recently released just a couple of months ago.

We can turn BINs into BBLs with a high level of accuracy by using New York City's [geoclient API](https://developer.cityofnewyork.us/api/geoclient-api). This seems to handle the construction aspect of 

An alternative option would be to modify `co_reader` to read `BIN` information off of the same certificate of occupancy listing pages that we currently use to get those. However, we did not take this route.

First we test this pipeline, using the first of the samples we have generated.

-->

Let's fetch a sample to work with while the data is getting sourced elsewhere.

In [12]:
r = requests.get('https://raw.githubusercontent.com/ResidentMario/nyc-active-construction-sites/master/active_sample_1.csv')
with open("active sample 1.csv", "w") as f:
    f.write(r.text)
del r

In [16]:
nb_active_sample = pd.read_csv("active sample 1.csv", error_bad_lines=False, encoding='latin1', index_col=0)

Match the cases of the column names so that `merge` can find them correctly.

In [21]:
nb_active_sample.columns = [column.title() for column in nb_active_sample.columns]

Match the borough acronyms. Boroughs are full name all-caps in the sample data (`BROOKLYN` etc.), but are standard two-letter codes in MapPLUTO (`BK` etc.).

In [29]:
borough_mapping = {
    'BK': 'BROOKLYN',
    'QN': 'QUEENS',
    'SI': 'STATEN ISLAND',
    'MN': 'MANHATTAN',
    'BX': 'BRONX'
                  }

brooklyn['Borough'] = brooklyn['Borough'].apply(lambda b: borough_mapping[b])

Finally we match the data and switch to a `GeoDataFrame`.

In [38]:
import geopandas as gpd
import mplleaflet

In [47]:
sample_brooklyn_lots = gpd.GeoDataFrame(pd.merge(nb_active_sample, brooklyn, how='inner', on=['Borough', 'Block', 'Lot']))

The lot coordinates need to be reprojected. The root projection below is the Long Island standard grid used by the PLUTO dataset, the target is simple latitude/longitude.

In [52]:
sample_brooklyn_lots.crs = {u'lon_0': -74, u'datum': u'NAD83', u'y_0': 0, u'no_defs': True, u'proj': u'lcc', u'x_0': 300000, u'units': u'us-ft', u'lat_2': 41.03333333333333, u'lat_1': 40.66666666666666, u'lat_0': 40.16666666666666}

In [57]:
sample_brooklyn_lots = sample_brooklyn_lots.to_crs({'proj':'longlat', 'ellps':'WGS84', 'datum':'WGS84'})

In [58]:
fig = sample_brooklyn_lots.plot()
mplleaflet.show()

Verdict: individual buildings are too small for their footprints to appear visible at meaningful resolution on a map like, and the basemap is too busy. With some further configuration, and accounting for the fact there will be many more of these buildings in the actual set, I believe that the map may still be viable, however. I'm unsure. A simpler pin map may well work better in this scenario.

Going forward there's nothing stopping us from using both.

### GeoJSON test

It was pointed out that there is a GeoJSON version of this data, but the GeoJSON data has a strangely small size. Taking a look:

In [ ]:
g

requests.get("http://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/MAPPLUTO/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=geojson")